# Импорт необходимых бибилиотек

In [27]:
!git clone https://github.com/Balaje/OpenCV.git

import os
from tqdm import tqdm
from time import time
from random import randint as rndm

import cv2

import mediapipe as mp

import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import torch.nn as nn

import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

from sklearn.utils import shuffle


Cloning into 'OpenCV'...


# Настройка модели для распознавания жестов

## Настраиваем трансформеры

In [16]:
# Задаем переменную, в которой хранятся названия папок, который и будут в дальнейшем нашими лэйблами
labels_original = ['01_palm', '02_l', '03_fist', '04_fist_moved', '05_thumb', '06_index', '07_ok', '08_palm_moved', '09_c', '10_down']

# Создаем транформер для обычной обработки изображения
transform_default = transforms.Compose([transforms.Pad(padding=(0, 200, 0, 200),    # добавление полей сверху и снизу
                                                       fill=(0),                    # черным цветов
                                                       padding_mode='constant'),    # заполнение цветом переданным в fill
                                        transforms.CenterCrop(400),                 # обрезка по центру до квадрата 400 на 400
                                        transforms.Resize((256,256)),               # уменьшаем размер изображений
                                        transforms.Grayscale(),                     # переводим в оттенки серого
                                        transforms.ToTensor()])                     # преобразуем в тензор
# Создаем трансформер для увеличения изначального объема изображений
transform_augmentation = transforms.Compose([transform_default,                                   # за основну берем дефолтный трансформ
                                             transforms.RandomRotation(degrees = rndm(5, 30)),    # случайный поворот от 5 до 30 градусов
                                             transforms.RandomHorizontalFlip(),                   # случайное отражение по горизонтали
                                             transforms.ColorJitter(                              # добавление случайных изменений
                                                brightness=rndm(0, 9)/10,                         # яркости
                                                contrast=rndm(0, 9)/10)                           # контраста
                                             ])

## Подготавливаем данные

In [ ]:
# Создаем список в котором содержаться датасеты с изображениями из всех папок, пропущенных через дефолтный трансформер
datasets = [torchvision.datasets.ImageFolder(root='leapGestRecog/'+people, transform=transform_default) for people in tqdm(os.listdir('leapGestRecog'))]
# Добавляем теже изображения, но немного измененные при помощи второго трансформера
datasets.extend([torchvision.datasets.ImageFolder(root='leapGestRecog/'+people, transform=transform_augmentation) for people in tqdm(os.listdir('leapGestRecog'))])
# Объединяем это все в единый датасет
dataset = torch.utils.data.ConcatDataset(datasets)
# Разбиваем на тренировочну, валидационную и тестовую выборки
train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    dataset, 
     [int(len(dataset) * 0.7),    # Тренировочная выборка
      int(len(dataset) * 0.2),    # Валидационная выборка
      int(len(dataset) * 0.1)],   # Тестовая выборка
  generator=torch.Generator().manual_seed(42))
# Задаем загрузчики данных
get_loader = lambda data: DataLoader(data, batch_size=16, shuffle=True, num_workers = 2)
batch_size = 16
train_loader, val_loader, test_loader = get_loader(train_dataset), get_loader(val_dataset), get_loader(test_dataset)

## Прописываем архитектуру модели
В данном случае, в процессе поиска различных вариантов, было выявленно, что эти две модели справляются довольно хорошо с поставленной задачей. В качестве эксперимента было принято решение объединить их. В дальнейшем ни одна другая модель не справлялась лучше, чем этот вариант. 

In [17]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 61 * 61, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 61 * 61)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

class New_Model(nn.Module):
    def __init__(self):
        super(New_Model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.bn2 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 61 * 61, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(p=0.5)

    def forward(self, x):
        x = self.pool(self.bn1(nn.functional.leaky_relu(self.conv1(x))))
        x = self.pool(self.bn2(nn.functional.leaky_relu(self.conv2(x))))
        x = x.view(-1, 64 * 61 * 61)
        x = nn.functional.leaky_relu(self.fc1(x))
        x = self.dropout(x)
        x = nn.functional.leaky_relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

class CombinedModel(nn.Module):
    def __init__(self, model1, model2):
        super(CombinedModel, self).__init__()
        self.model1 = model1
        self.model2 = model2
        for param in self.model1.parameters():
            param.requires_grad = False
        out_features = model1.fc3.out_features
        self.fc3 = nn.Linear(2*out_features, out_features)

    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        out = torch.cat((out1, out2), 1)
        out = self.fc3(out)
        return out

## Собственная функция точности модели

In [ ]:
# Получает модель для тестирования, возвращает среднее значения удаленности истинного значения от предсказанного
def check(m):
  r = []

  # Оценка насколько правильное значения близко к предсказанному
  # Идеально когда 0
  my_accuracy = lambda pred, true: pred.max() - pred[0][true]

  for name in os.listdir('my_test_image/'):

    # Загружаем изображение
    test_image_path = f'my_test_image/{name}'
    # Загружаем руку модулем подходящим для модели по поиску координат руки на изображении
    img = cv2.imread(test_image_path)
    # Инициализация MediaPipe Hands
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands()
    # Обнаружение ключевых точек на руке на изображении, которое мы трансформировали в RGB формат из дефолтного для CV2 BGR формата
    results = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    # Прописываем координнаты точек по осям X и Y в свои списки
    x_list = [p.x for p in results.multi_hand_landmarks[0].landmark]
    y_list = [p.y for p in results.multi_hand_landmarks[0].landmark]
    # Определяем координаты квадрата, в котором расположена рука
    x = int(min(x_list)*img.shape[1])-20
    y = int(min(y_list)*img.shape[0])-20
    w = int(max(x_list)*img.shape[1])-int(min(x_list)*img.shape[1])+40
    h = int(max(y_list)*img.shape[0])-int(min(y_list)*img.shape[0])+40

    # Загружаем руку модулем, удобным для нас
    test_image = Image.open(test_image_path)
    # Обрезаем изображения так, чтобы осталась одна только рука
    test_image_hand = test_image.crop((x-20, y-20, x+w+40, y+h+40))
    # Подготовка изображения для модели
    image_tensor = transform_default(test_image_hand).unsqueeze(0)
    # Получаем предсказания
    pred = m(image_tensor)
    # Добавляем их в список результатов
    r.append(float(my_accuracy(pred, [l[3:] for l in labels_original].index(name.split('.')[0]))))
    pred = pred.argmax()
  return sum(r)/len(r)

## Обучение модели

In [ ]:
# Определение устройства вычислений
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Инициализация модели и оптимизатора
model = CombinedModel(Model().to(device), New_Model().to(device))
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.2) # по результатам тестов лучшее сочетание

# Задаем количество эпох для обучения модели
num_epochs = 20
# Запускаем цикл обучения
for epoch in range(num_epochs):
  # Инициализация переменной потерь для текущей эпохи
  running_loss = 0.0
  # Инициализация списков для хранения значений потерь и метрик для последующей оценки модели
  loss_list, acc_list = []
  # Переводим модель в режим обучения
  model.train()
  # Создаем индикатор прогресса обучения с общим числом итераций равным размеру обучающего набора данных
  with tqdm(total=len(train_loader)) as pbar:
      # Цикл по данным из обучающего набора
      for data in train_loader:
          # Обнуление градиентов оптимизатора
          optimizer.zero_grad()
          # Получаем отдельно изображение и его лэйбл
          inputs, labels = data[0].to(device), data[1].to(device)
          # Прямой проход через модель
          outputs = model(inputs)
          # Вычисление функции потерь
          loss = criterion(outputs, labels)
          # Вычисление градиентов
          loss.backward()
          # Обновление весов модели
          optimizer.step()
          # Добавление текущей потери к общей потере
          running_loss += loss.item()
          # Вычисление точности для текущего пакета данных
          acc = 100 * sum((np.array([int(o.argmax()) for o in outputs]) == np.array(labels)).astype(int)) / len(outputs)
          # Обновление прогресс бара информацией о том сколько эпох пройдено, потерях и точности
          pbar.set_description(f'Epoch {epoch + 1}/{num_epochs}')
          pbar.postfix = (f'| Loss: {loss:.3f} | Accuracy: {acc:.2f}%')
          pbar.update()
  # Вывод значения кросс-энтропии для текущей эпохи
  print(f'\nCrossEntropyLoss (train): {running_loss / len(train_loader):.5f}')

  # Переводим модель в режим валидации
  model.eval()
  # Инициализация переменной потерь для валидации
  running_loss = 0.0
  # Инициализация переменных для подсчета точности модели на тестовой выборке
  correct, total = 0, 0
  # Выключаем отслеживание градиентов для валидации
  with torch.no_grad():
      # Цикл по данным из тестового набора
      for data in tqdm(test_loader):
          # Распределение входных данных и меток по устройству (обычно GPU)
          inputs, labels = data[0].to(device), data[1].to(device)
          # Прямой проход через модель
          outputs = model(inputs)
          # Получение метки с максимальным значением выхода
          _, predicted = torch.max(outputs.data, 1)
          # Инкрементирование общего количества
          total += labels.size(0)
          # Инкрементирование количества правильно классифицированных объектов
          correct += (predicted == labels).sum().item()
  # Вывод точности на данных из набора
  print(f'Accuracy (val): %d %% \n' % (100 * correct / total))
  # Вывод точности на данных из собственного набора
  print(f'Accuracy (my_test): {check(model)}')

  # Сохранение текущей модели на диск
  torch.save({'transformer': transform_default,
              'model_state_dict': model.state_dict()},
            f'Models/model_e{epoch}.pth')

### Результаты обучения самых различных вариантов (не только этого)

**Version 1**  
* image_size = (16, 16)  
* batch_size = 4  
* num_epoch = 10  
* Обучена обсолютно на всех данных что есть. 

Точность - 99.99%  
Скорость обучения - 21 минут 45 секунды

--- 

**Version 2**  
* image_size = (16, 16)  
* batch_size = 16  
* num_epoch = 10  
* Изначальные данные были разделены на тренировочную и тестовую выборки (соотношение 80/20).

Точность - 99.85%  
Функция потерь - 0.01648  
Скорость обучения - 13 минут 22 секунд

---

**Version 2.1**
* Изменения трансформера, модель теперь принимает чернобелые изображения размеров 250*250  

Epoch 1/10:   
* CrossEntropyLoss (train): 2.30379  
* Accuracy (val): 9 % 

Epoch 10/10:  
* CrossEntropyLoss (train): 2.28108  
* Accuracy (val): 15 %  

Точность модели (test): 9 %

---

**Version 2.2**  
* из агуметации убраны: поврот по вертикали, увеличения красочности, насыщенности  
* в аугментацию добавлены: доведение до квадрата путем заполнения пустот черным цветом (чтобы не размазывало изображения), обрезка изображения по центру рандомизация поворот, яркости и контрастности.
* обучена на лучшей первой версии модели  

Epoch 1/20: 
* CrossEntropyLoss (train): 2.27711
* Accuracy (val): 19 % 

Epoch 10/20:
* CrossEntropyLoss (train): 1.37242
* Accuracy (val): 61 %

Epoch 20/20:
* CrossEntropyLoss (train): 0.44998
* Accuracy (val): 88 %

Точность модели (test): 10 %

---

**Version 3 (final)**
* Выбрана лучшая из всех модель для дообучения
* Написана еще одна модель и объединена с лучшей  
* Была написанна функция, которая возвращает среднюю дальность предсказанного класса от истинного класса

Epoch 1/1: 
* CrossEntropyLoss (train): 2.01468
* Accuracy (val): 62 %
* Accuracy (test): 9 %

Epoch 2/20:
* CrossEntropyLoss (train): 1.99220
* Accuracy (val): 62 %
* Accuracy (my_test): 0,6804 

Epoch 3/20: 
* CrossEntropyLoss (train): 1.13265
* Accuracy (val): 83 %
* Accuracy (my_test): 0,9211

Epoch 4/20: 
* CrossEntropyLoss (train): 0.88938
* Accuracy (val): 87 %
* Accuracy (my_test): 1.01267

Epoch 5/20:
* CrossEntropyLoss (train): 0.65368
* Accuracy (val): 93 %
* Accuracy (my_test): 1.2956

Epoch 6/20: 
* CrossEntropyLoss (train): 0.50916
* Accuracy (val): 95 %
* Accuracy (my_test): 1.36023

--- 

**Вывод**
По результам было принято решение не обучать модель до конца, а остановиться на второй эпохе, т.к. в дальнейшем она начинала переобучаться и на реальных данных всем изображениям прописывала один, максимум два типа.

## Загружаем модель, которая показала лучшие результаты
Модель которая дошла до 3 эпоих показал себя лучше всего, было принято решение взять ее за основу.  
palm - распознает примерно в 50%  
ok - распознает в 100%  
down - если ближе к камере, но приммерно в 25%  
index - если повернуть руку примерно на 45 градусов, но примерно в 25%  
все жесты в целом лучше распознаются если показывать их правой рукой  

In [20]:
model = CombinedModel(Model(), New_Model())
model.load_state_dict(state_dict = torch.load('model_e2.pth')['model_state_dict'])

<All keys matched successfully>

# Настройка модели для поиска лица на изображении

In [19]:
def find_face(img):
    # преобразование изображения в оттенки серого
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # создание экземпляра класса CascadeClassifier для поиска объектов
    haar_cascade = cv2.CascadeClassifier('C:\\PyProject\\Home_Work\\PyTorch\\Course_Work\\OpenCV\\haarcascades\\face.xml')
    # поиск лица на изображении
    face = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    # обводим найденное лицо
    for (x, y, w, h) in face:
      cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return img

# Основной код
Захват изображения с камеры, обработка всеми ранее прописанными функциями и моделями. Вывод на экране пользователю изображения с камеры, с найденным лицом, рукой и предсказанием типа жеста.

In [30]:
# Прописываем названия классов
labels = ['palm', 'l', 'fist', 'fist_moved', 'thumb', 'index', 'ok', 'palm_moved', 'c', 'down']
# Создаем датафрем для записис результатов
df = pd.DataFrame([[[], 0, 0] for _ in range(len(labels))], columns=['values', 'frequency', 'probability'], index=labels)

# Создаем объекты для обнаружения рук и рисования ключевых точек
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()
mp_drawing = mp.solutions.drawing_utils

# Создаем объект VideoCapture для захвата видео с веб-камеры
cap = cv2.VideoCapture(0)

# Проверяем успешность открытия видео устройства
if not cap.isOpened():
    print("Не удалось открыть видео устройство")
    exit()

# Запускаем бесконечный цикл для захвата и обработки изображения с камеры
while True:
    # Захватываем кадр из видео
    ret, frame = cap.read()
    # Проверяем успешность захвата кадра
    if not ret:
        print("Не удалось получить кадр")
        break
    frame = find_face(frame)
    # Преобразуем кадр из BGR в RGB и пытаемся обнаружить руки в кадре
    results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    cat_frame = np.array([[], []])
    # Начинаем работу с изображением если рука найдена, если нет, то происходит отчистка датафрема с результатами
    if results.multi_hand_landmarks:
        # отмечаем ключевые точки руки
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        # обрезка изображения по наивысшим точкам каждой оси с допуском в 40 пикселей
        x_list = [p.x for p in results.multi_hand_landmarks[0].landmark]
        y_list = [p.y for p in results.multi_hand_landmarks[0].landmark]
        h, w, c = frame.shape
        cat_frame = frame[int(min(y_list) * h) - 40:int(max(y_list) * h) + 40, int(min(x_list) * w) - 40:int(max(x_list) * w) + 40]
    else:
        df = pd.DataFrame([[[], 0, 0] for _ in range(len(labels))], columns=['values', 'frequency', 'probability'], index=labels)

    # если в каком-то из измерений массива изображения отсутсвуют значения, то происходит отчистка датафрема с результатами
    if 0 not in cat_frame.shape:
        # трансформируем изображение под модель
        input = transform_default(F.to_pil_image(cat_frame)).unsqueeze(0)
        # получаем предсказание от модели
        preds = model(input).detach().numpy()[0]
        
        # записываем список предсказний
        df.iloc[preds.argmax(), 0].append(preds.max())
        # записываем частоты выбора позиции
        df.iloc[preds.argmax(), 1] = len(df.iloc[preds.argmax(), 0])
        # записываем вероятность того что это именно это значение
        df.iloc[preds.argmax(), 2] = len(df.iloc[preds.argmax(), 0])/sum(df['frequency'])*100
        # получаем значения которое выбиралось чаще всего
        predict_label = df.loc[df['frequency'] == df['frequency'].max()]
        #  получаем значения с наибольшей вероятностью
        predict = predict_label.loc[predict_label['probability'] == predict_label['probability'].max()]
        # прописываем название лэйбла, есть частоту выбора и вероятность что это именно оно
        predict_text = f"{predict.index[0]}: {predict['frequency'].values[0]} - {predict['probability'].values[0]:.2f}%"

        # Создание фона для текста
        text_background = np.zeros((100, 500, 3), dtype=np.uint8)
        # Добавление predict_text
        cv2.putText(text_background, predict_text, (50, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        # Отображение окна
        cv2.imshow('Predict', text_background)
    else:
        df = pd.DataFrame([[[], 0, 0] for _ in range(len(labels))], columns=['values', 'frequency', 'probability'], index=labels)

    # Отображаем кадр с нарисованными точками в окне
    cv2.imshow('Webcam', cv2.flip(frame, 1))

    # Обработка нажатия клавиши 'q' для выхода из цикла
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Освобождаем ресурсы
cap.release()
cv2.destroyAllWindows()